In [ ]:
import time
from sklearn.metrics import roc_auc_score
import json
import os
import copy
import argparse
import numpy as np
import torch

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from modules.model import Model, Dataloader
from modules.utils import load_dataset, set_random_seeds, rescale
from modules.experiment import run_experiment
from modules.argument import TrainArguments

In [ ]:
def train(model = None, **train_args):
    args = TrainArguments(**train_args)
    # Setup torch
    device = torch.device(args.device if torch.cuda.is_available() else 'cpu')
    # Load dataset
    g, features, ano_label, _, _ = load_dataset(args.dataset)
    features = torch.FloatTensor(features)
    if args.batch_size == -1:
        features = features.to(device)
    g = g.to(device)
    dataloader = Dataloader(g, features, args.k, dataset_name=args.dataset)
    if not os.path.isdir("./ckpt"):
        os.makedirs("./ckpt")

    # Run the experiment
    seed = args.seed
    model, stats = run_experiment(args, seed, device, dataloader, ano_label, model)
    print("AUC: %.4f" % stats["AUC"])
    print("Time (Train): %.4fs" % stats["time_train"])
    print("Mem (Train): %.4f MB" % (stats["mem_train"] / 1024 / 1024))
    print("Time (Test): %.4fs" % stats["time_test"])
    print("Mem (Test): %.4f MB" % (stats["mem_test"] / 1024 / 1024))
    return model, stats

In [ ]:
model, stats = train(dataset="cora", lr=0.03, alpha=0.9, gamma=0.1, num_epoch=100, batch_size=1)